In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX


# --- 1. Load data from CSV file ---
# The CSV has one row of dates (MM/DD/YYYY) and one row of bill amounts
# Example:
# 7/1/2022, 8/1/2022, 9/1/2022, ...
# 120, 130, 125, ...
csv_path = '../data/actual_power_data.csv'

df_raw = pd.read_csv(csv_path, header=None)
date_strings = df_raw.iloc[0].values
bill_values = df_raw.iloc[1].values.astype(float)

# 2. Convert dates and create a time series
dates = pd.to_datetime(date_strings, format='%m/%d/%Y')
bills_series = pd.Series(bill_values, index=dates)

# 3. Fit SARIMAX model without exogenous variable
model = SARIMAX(bills_series,
                order=(1,1,1),
                seasonal_order=(1,1,1,12),
                enforce_stationarity=False,
                enforce_invertibility=False
                )
results = model.fit(disp=False)

# 4. Forecast next 12 months
future_index = pd.date_range(start=dates[-1] + pd.offsets.MonthBegin(), periods=12, freq='MS')
forecast = results.get_forecast(steps=12)
forecast_mean = forecast.predicted_mean
forecast_ci = forecast.conf_int()

# 5. Plot the forecast
plt.figure(figsize=(12,6))
plt.plot(bills_series, label='Observed Bills', color='blue')
plt.plot(future_index, forecast_mean, label='Forecasted Bills', color='red')
plt.fill_between(future_index, forecast_ci.iloc[:,0], forecast_ci.iloc[:,1],
                 color='pink', alpha=0.3)

plt.title('Power Bill Forecast (No Exogenous Variable)')
plt.xlabel('Date')
plt.ylabel('Bill Amount ($)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# 6. Print forecasted bills
print("Forecasted power bills without temperature:")
for date, value in zip(future_index, forecast_mean):
    print(f"{date.strftime('%B %Y')}: ${value:.2f}")